<center><h1>Mínimos cuadrados ordinarios</h1></center>

## 1. Introducción.
En la última lección, exploramos una técnica iterativa para el ajuste de modelos denominada descenso de gradiente. El algoritmo de descenso de gradiente requiere múltiples iteraciones para converger en los valores óptimos de los parámetros y el número de iteraciones depende en gran medida de los valores iniciales de los parámetros y la tasa de aprendizaje que seleccionemos.

A continuación, exploraremos una técnica llamada **estimación por mínimos cuadrados ordinarios** o estimación OLS para abreviar. A diferencia del descenso de gradiente, la estimación OLS proporciona una fórmula clara para calcular directamente los valores óptimos de los parámetros que minimizan la función de costo. Para comprender la estimación de OLS, primero debemos enmarcar nuestro problema de regresión lineal en forma de matriz. Hemos trabajado principalmente con la siguiente forma del modelo de regresión lineal:

$\hat{y} = a_0 + a_1x_1 + a_2x_2 + ... + a_nx_n$

Si bien esta forma representa la relación entre las características ($x_1$ a $x_n$) y la columna de destino ($y$) cuando hay solo unos pocos valores de parámetros, no se escala bien cuando tenemos cientos de parámetros. La notación matricial nos permite representar y razonar mejor sobre un sistema lineal con muchas variables. Con eso en mente, así es como se ve la forma matricial de nuestro modelo de regresión lineal:

$Xa  =  \hat{y}$

Donde $X$ es una matriz que representa las columnas del conjunto de entrenamiento que usa nuestro modelo, $a$ es un vector que representa los valores de los parámetros, y $\hat{y}$ es el vector de predicciones. Aquí hay un diagrama con algunos valores de muestra para cada uno: 

![matrix](figs/matrix_form.svg)

Ahora que hemos adquirido una comprensión de la representación matricial del modelo de regresión lineal, echemos un vistazo a la fórmula de estimación OLS que da como resultado el vector óptimo $a$:


$a = (X^TX)^{-1} X^Ty$

Comencemos calculando la estimación OLS para encontrar los mejores parámetros para un modelo usando las siguientes características:

```python
features = ['Wood Deck SF', 'Fireplaces', 'Full Bath', '1st Flr SF', 'Garage Area',
       'Gr Liv Area', 'Overall Qual']
```

Nos sumergiremos en la derivación matemática de la técnica de estimación OLS. Es importante tener en cuenta que lo más probable es que nunca implemente esta técnica en una función de ciencia de datos y, en su lugar, use una implementación existente y eficiente (scikit-learn usa OLS cuando llama a `fit()` en una instancia de [LinearRegression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html)).

### Ejercicio

- Cree un dataframe, `X`, donde:
    - Su número de filas es el mismo que el del `train`
    - La primera columna se llama `bias` y se completa con el número `1` en todo
    - Las siguientes columnas son las de `features` de `train`, en el mismo orden.
- Seleccione la columna `SalePrice` del conjunto de entrenamiento y asígnela a `y`.
- Utilice la fórmula de estimación OLS para obtener los valores de parámetros óptimos. Asigne la estimación a la variable `ols_estimation`

In [12]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

In [16]:
data = pd.read_csv('AmesHousing.txt', delimiter='\t')
train = data[:1460].copy()
test = data[1460:].copy()

features = ['Wood Deck SF', 'Fireplaces', 'Full Bath', '1st Flr SF', 'Garage Area',
       'Gr Liv Area', 'Overall Qual']

In [20]:
# print(['Order'] + features)
X = train[['Order'] + features].copy()
X.rename(columns={'Order':'bias'}, inplace=True)
# X.columns
print(X.shape)

(1460, 8)


In [22]:
X['bias'] = 1
X

,bias,Wood Deck SF,Fireplaces,Full Bath,1st Flr SF,Garage Area,Gr Liv Area,Overall Qual
0,1,210,2,1,1656,528.0,1656,6
1,1,140,0,1,896,730.0,896,5
2,1,393,0,1,1329,312.0,1329,6
3,1,0,2,2,2110,522.0,2110,7
4,1,212,1,2,928,482.0,1629,5
...,...,...,...,...,...,...,...,...
1455,1,0,1,2,992,839.0,1865,7
1456,1,0,1,2,1290,570.0,2161,7
1457,1,224,2,2,1004,660.0,1852,8
1458,1,192,1,2,1153,541.0,2031,7


In [23]:
y = train['SalePrice'].copy()

print(y)

In [35]:
# OLS formula

ols_estimation = np.linalg.inv(X.T.dot(X)).dot(X.T).dot(y)

print(ols_estimation.reshape((-1, 1)))

[[-1.12764871e+05]
 [ 3.78815268e+01]
 [ 7.08698430e+03]
 [-2.22197281e+03]
 [ 4.31853639e+01]
 [ 6.48808564e+01]
 [ 3.87112549e+01]
 [ 2.45531837e+04]]


## 2. Función de Costo.
A diferencia del descenso de gradiente, la estimación OLS proporciona lo que se conoce como una **solución de forma cerrada** al problema de encontrar los valores óptimos de los parámetros. Una solución de forma cerrada es aquella en la que una solución se puede calcular aritméticamente con una cantidad predecible de operaciones matemáticas. El descenso de gradiente, por otro lado, es un enfoque algorítmico que puede requerir un número diferente de iteraciones (y, por lo tanto, un número diferente de operaciones matemáticas) en función de los valores de los parámetros iniciales, la tasa de aprendizaje, etc. Si bien el enfoque es diferente, ambos Las técnicas comparten el objetivo de alto nivel de minimizar la función de costo.

Antes de que podamos sumergirnos en cómo se representa la función de costo en forma de matriz, comprendamos cómo se representa el error. Porque el error es la diferencia entre las predicciones hechas usando el modelo $\hat{y}$  y las etiquetas reales $y$ , se representa como un vector. La letra griega para $\epsilon$ (épsilon) se utiliza a menudo para representar el vector de error:

$\epsilon =  \hat{y} - y$

Podemos basarnos en esto para definir $y$

$y = Xa - \epsilon$

Aunque esto se parece mucho a la ecuación matricial de $Ax = b$ , tenemos 2 incógnitas (el vector $a$   y el vector $\hat{y}$). Estamos buscando un modelo, representado usando el vector de parámetros $a$ , que minimizará el error cuadrático medio entre las etiquetas, $y$, y las predicciones, $\hat{y}$ . Dicho de otra manera, la función de costo es este error cuadrático medio.

Así es como se ve la función de costo en forma de matriz:

$J(a) = \dfrac{1}{n} (Xa - y)^T(Xa - y)$

## 3. Derivada De La Función De Costo.

Comprender la derivación de la función de costo requiere cierta familiaridad con el [cálculo matricial](https://en.wikipedia.org/wiki/Matrix_calculus), que es una notación específica para aplicar conceptos de cálculo a matrices. Se recomienda [leer el siguiente blog](https://eli.thegreenplace.net/2015/the-normal-equation-and-matrix-calculus/).

Aquí está la derivada de la función de costo:

$\frac{dJ(a)}{da} = 2X^TXa - 2X^Ty$

Para encontrar el vector $a$ que minimiza la función de costo $J(a)$ , necesitamos igualar la derivada a $0$   y resolver para $a$ :

$2X^TXa - 2X^Ty = 0$

Movamos el segundo término al lado derecho y dividamos ambos lados por $2$:

$X^TXa = X^Ty$

Nuestro objetivo es aislar $a$, el vector de parámetros. El último paso que debemos realizar es "dividir" $X^TX$ del lado izquierdo.

Podemos "dividir" los términos de la matriz calculando el inverso. Podemos cancelar $A$ de la siguiente ecuación $A x = b$ multiplicando ambos lados por el inverso $A^{−1}Ax = A^{−1}b$ . Esto nos deja con $x = A^{−1}b$ .

Para cancelar $X^TX$ del lado izquierdo, necesitamos calcular el inverso y multiplicarlo por ambos lados. Ahora nos quedamos con la fórmula de estimación OLS:

Para cancelar $X^TX$ del lado izquierdo, necesitamos calcular el inverso y multiplicarlo por ambos lados. Ahora nos quedamos con la fórmula de estimación OLS:  $$a = (X^TX)^{-1}X^Ty$$
### Ejercicio

## 4. Descenso de gradiente frente a mínimos cuadrados ordinarios.

Ahora que hemos explorado muchas de las matemáticas que subyacen a la estimación OLS, comprendamos sus limitaciones. La mayor limitación es que la estimación OLS es computacionalmente costosa cuando los datos son grandes. Esto se debe a que calcular una matriz inversa tiene una complejidad computacional de aproximadamente O(n^3). Puede leer más sobre la complejidad computacional de la matriz inversa y otras operaciones de matriz comunes [aquí](https://en.wikipedia.org/wiki/Computational_complexity_of_mathematical_operations#Matrix_algebra).

OLS se usa comúnmente cuando la cantidad de elementos en el conjunto de datos (y, por lo tanto, la matriz que está invertida) es inferior a unos pocos millones de elementos. En conjuntos de datos más grandes, se usa el descenso de gradiente porque es mucho más flexible. Para muchos problemas prácticos, podemos establecer un valor 
de umbral (o un número determinado de iteraciones) y usar una solución "suficientemente buena". Esto es especialmente útil al iterar y probar diferentes funciones en nuestro modelo.


## 5. Conclusión.

Exploramos una solución de forma cerrada para ajustar un modelo de regresión lineal llamado estimación OLS. Exploramos algo de la intuición detrás de las matemáticas para esta técnica y terminamos explorando su complejidad computacional. Posteriormente, exploraremos cómo limpiar algunas de las funciones restantes en el conjunto de entrenamiento para usarlas en nuestro modelo.